In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!unzip -q /content/TL_csv.zip -d /content/drive/MyDrive/건양대의료인공지능_실습/Travel_recommendation

In [3]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
pd.options.display.max_columns = 20
np.random.seed(2023)
np.set_printoptions(precision=4, suppress=True)
import matplotlib.pyplot as plt
import matplotlib
plt.rc('figure', figsize=(8, 4))

In [ ]:
# ai hub
# https://aihub.or.kr/aihubdata/data/list.do?pageIndex=1&currMenu=&topMenu=&dataSetSn=&srchdataClCode=DATACL001&searchKeyword=%EC%97%AC%ED%96%89&srchDetailCnd=DETAILCND001&srchOrder=ORDER001&srchPagePer=20

In [4]:
!wget -q https://raw.githubusercontent.com/kairess/toy-datasets/master/tn_travel_%E1%84%8B%E1%85%A7%E1%84%92%E1%85%A2%E1%86%BC_A.csv
!wget -q https://raw.githubusercontent.com/kairess/toy-datasets/master/tn_traveller_master_%E1%84%8B%E1%85%A7%E1%84%92%E1%85%A2%E1%86%BC%E1%84%80%E1%85%A2%E1%86%A8%20Master_A.csv
!wget -q https://raw.githubusercontent.com/kairess/toy-datasets/master/tn_visit_area_info_%E1%84%87%E1%85%A1%E1%86%BC%E1%84%86%E1%85%AE%E1%86%AB%E1%84%8C%E1%85%B5%E1%84%8C%E1%85%A5%E1%86%BC%E1%84%87%E1%85%A9_A.csv

In [12]:
# 데이터셋 로드
df_place = pd.read_csv('/content/drive/MyDrive/건양대의료인공지능_실습/Travel_recommendation/tn_visit_area_info_방문지정보_A.csv')
df_place.head()

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,VISIT_START_YMD,VISIT_END_YMD,ROAD_NM_ADDR,LOTNO_ADDR,X_COORD,Y_COORD,...,POI_NM,RESIDENCE_TIME_MIN,VISIT_AREA_TYPE_CD,REVISIT_YN,VISIT_CHC_REASON_CD,LODGING_TYPE_CD,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION,SGG_CD
0,2210300006,a_a015688,5,프로방스마을,2022-10-30,2022-10-30,경기 파주시 탄현면 새오리로 77,경기 파주시 탄현면 성동리 82-1,126.685239,37.790405,...,NaN,60.0,7,Y,9.0,NaN,4.0,4.0,4.0,NaN
1,2208050001,a_a000164,1,친구/친지집,2022-08-05,2022-08-05,NaN,경기도 안산시 단원구 와동,NaN,NaN,...,NaN,NaN,22,NaN,NaN,NaN,NaN,NaN,NaN,41273.0
2,2210030002,a_a004293,2,병점역 1호선,2022-10-03,2022-10-03,경기 화성시 떡전골로 97,경기 화성시 진안동 824-1,127.033200,37.206753,...,병점역,30.0,9,Y,6.0,NaN,4.0,5.0,5.0,NaN
3,2208200003,a_a000491,12,더현대서울,2022-08-20,2022-08-20,서울 영등포구 여의대로 108,서울 영등포구 여의도동 22,126.928446,37.525872,...,NaN,180.0,4,N,2.0,NaN,5.0,5.0,5.0,NaN
4,2208110007,a_a000172,13,강릉중앙시장,2022-08-11,2022-08-11,강원 강릉시 금성로 21,강원 강릉시 성남동 50,128.898624,37.754025,...,신흥수산,30.0,4,N,1.0,NaN,4.0,4.0,4.0,NaN


In [15]:
df_travel = pd.read_csv('/content/drive/MyDrive/건양대의료인공지능_실습/Travel_recommendation/tn_travel_여행_A.csv')
df_travel.head()

,TRAVEL_ID,TRAVEL_NM,TRAVELER_ID,TRAVEL_PURPOSE,TRAVEL_START_YMD,TRAVEL_END_YMD,MVMN_NM,TRAVEL_PERSONA,TRAVEL_MISSION,TRAVEL_MISSION_CHECK
0,a_a014262,A03,a014262,1;11;2;22;,2022-10-29,2022-10-30,자가용,NaN,1;11;2;22;,2;1;22
1,a_a006728,A03,a006728,21;24;5;22;26;27;,2022-10-10,2022-10-11,자가용,NaN,21;24;5;22;26;27;,6;1;22
2,a_a015745,A02,a015745,1;2;6;,2022-10-31,2022-11-03,대중교통 등,NaN,1;2;6;,13;3;28
3,a_a010544,A03,a010544,21;22;24;26;,2022-11-01,2022-11-02,자가용,NaN,21;22;24;26;,1;22;26
4,a_a012058,A03,a012058,1;4;10;12;21;22;26;,2022-11-05,2022-11-06,대중교통 등,수도권 외 거주 39세 이하 커플,1;4;10;12;21;22;26;,1;21;22


In [16]:
df_travel.info() #3200, 10

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3200 entries, 0 to 3199
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   TRAVEL_ID             3200 non-null   object
 1   TRAVEL_NM             3200 non-null   object
 2   TRAVELER_ID           3200 non-null   object
 3   TRAVEL_PURPOSE        3200 non-null   object
 4   TRAVEL_START_YMD      3200 non-null   object
 5   TRAVEL_END_YMD        3200 non-null   object
 6   MVMN_NM               2484 non-null   object
 7   TRAVEL_PERSONA        1040 non-null   object
 8   TRAVEL_MISSION        3200 non-null   object
 9   TRAVEL_MISSION_CHECK  3200 non-null   object
dtypes: object(10)
memory usage: 250.1+ KB


In [17]:
df_traveler = pd.read_csv('/content/drive/MyDrive/건양대의료인공지능_실습/Travel_recommendation/tn_traveller_master_여행객_Master_A.csv')
df_traveler.head()

,TRAVELER_ID,RESIDENCE_SGG_CD,GENDER,AGE_GRP,EDU_NM,EDU_FNSH_SE,MARR_STTS,FAMILY_MEMB,JOB_NM,JOB_ETC,...,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_STATUS_RESIDENCE,TRAVEL_STATUS_DESTINATION,TRAVEL_STATUS_ACCOMPANY,TRAVEL_STATUS_YMD,TRAVEL_MOTIVE_1,TRAVEL_MOTIVE_2,TRAVEL_MOTIVE_3,TRAVEL_COMPANIONS_NUM
0,b015583,41,여,20,6,1.0,1.0,5,3.0,NaN,...,1,7,경기도,충남,2인 여행(가족 외),2022-10-29~2022-10-30,1,NaN,NaN,1
1,a001105,30,남,30,6,1.0,1.0,2,3.0,NaN,...,6,5,대전광역시,경기,3인 이상 여행(가족 외),2022-09-03~2022-09-04,3,NaN,NaN,7
2,a001673,41,여,30,4,1.0,2.0,3,NaN,1.0,...,2,7,경기도,서울,자녀 동반 여행,2022-09-18~2022-09-19,10,7.0,1.0,2
3,a017690,41,여,30,6,1.0,1.0,3,3.0,NaN,...,4,5,경기도,서울,3인 이상 여행(가족 외),2022-11-12~2022-11-12,3,1.0,2.0,3
4,c006666,28,여,30,6,1.0,1.0,3,13.0,NaN,...,5,5,인천광역시,경기,나홀로 여행,2022-10-18~2022-10-18,1,7.0,2.0,0


In [18]:
# 데이터 병합
df = pd.merge(df_place, df_travel, on='TRAVEL_ID', how='left')
df = pd.merge(df, df_traveler, on='TRAVELER_ID', how='left')
df.head()

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,VISIT_START_YMD,VISIT_END_YMD,ROAD_NM_ADDR,LOTNO_ADDR,X_COORD,Y_COORD,...,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_STATUS_RESIDENCE,TRAVEL_STATUS_DESTINATION,TRAVEL_STATUS_ACCOMPANY,TRAVEL_STATUS_YMD,TRAVEL_MOTIVE_1,TRAVEL_MOTIVE_2,TRAVEL_MOTIVE_3,TRAVEL_COMPANIONS_NUM
0,2210300006,a_a015688,5,프로방스마을,2022-10-30,2022-10-30,경기 파주시 탄현면 새오리로 77,경기 파주시 탄현면 성동리 82-1,126.685239,37.790405,...,6.0,6.0,경기도,경기,나홀로 여행,2022-10-30~2022-10-31,1.0,7.0,10.0,0.0
1,2208050001,a_a000164,1,친구/친지집,2022-08-05,2022-08-05,NaN,경기도 안산시 단원구 와동,NaN,NaN,...,3.0,5.0,경기도,경기,2인 여행(가족 외),2022-08-05~2022-08-06,3.0,2.0,1.0,1.0
2,2210030002,a_a004293,2,병점역 1호선,2022-10-03,2022-10-03,경기 화성시 떡전골로 97,경기 화성시 진안동 824-1,127.033200,37.206753,...,7.0,2.0,경기도,경기,나홀로 여행,2022-10-03~2022-10-03,6.0,2.0,1.0,0.0
3,2208200003,a_a000491,12,더현대서울,2022-08-20,2022-08-20,서울 영등포구 여의대로 108,서울 영등포구 여의도동 22,126.928446,37.525872,...,5.0,7.0,울산광역시,서울,2인 가족 여행,2022-08-19~2022-08-20,1.0,3.0,7.0,1.0
4,2208110007,a_a000172,13,강릉중앙시장,2022-08-11,2022-08-11,강원 강릉시 금성로 21,강원 강릉시 성남동 50,128.898624,37.754025,...,2.0,4.0,광주광역시,서울,나홀로 여행,2022-08-10~2022-08-13,1.0,7.0,2.0,0.0


In [19]:
df[df['TRAVEL_ID'] == 'a_a015688']

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,VISIT_START_YMD,VISIT_END_YMD,ROAD_NM_ADDR,LOTNO_ADDR,X_COORD,Y_COORD,...,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_STATUS_RESIDENCE,TRAVEL_STATUS_DESTINATION,TRAVEL_STATUS_ACCOMPANY,TRAVEL_STATUS_YMD,TRAVEL_MOTIVE_1,TRAVEL_MOTIVE_2,TRAVEL_MOTIVE_3,TRAVEL_COMPANIONS_NUM
0,2210300006,a_a015688,5,프로방스마을,2022-10-30,2022-10-30,경기 파주시 탄현면 새오리로 77,경기 파주시 탄현면 성동리 82-1,126.685239,37.790405,...,6.0,6.0,경기도,경기,나홀로 여행,2022-10-30~2022-10-31,1.0,7.0,10.0,0.0
7060,2210300002,a_a015688,7,다온무인텔,2022-10-30,2022-10-31,경기 파주시 탄현면 성동로 7-25,경기 파주시 탄현면 성동리 666-3,126.686597,37.780606,...,6.0,6.0,경기도,경기,나홀로 여행,2022-10-30~2022-10-31,1.0,7.0,10.0,0.0
8788,2210300005,a_a015688,4,헤이리예술마을,2022-10-30,2022-10-30,NaN,경기 파주시 탄현면 법흥리 1652-239,126.698953,37.788174,...,6.0,6.0,경기도,경기,나홀로 여행,2022-10-30~2022-10-31,1.0,7.0,10.0,0.0
10467,2210310002,a_a015688,9,집,2022-10-31,2022-11-01,NaN,NaN,NaN,NaN,...,6.0,6.0,경기도,경기,나홀로 여행,2022-10-30~2022-10-31,1.0,7.0,10.0,0.0
13659,2210310001,a_a015688,8,아이노스 출판단지점,2022-10-31,2022-10-31,경기 파주시 문발로 142,경기 파주시 문발동 531-1,126.686457,37.707170,...,6.0,6.0,경기도,경기,나홀로 여행,2022-10-30~2022-10-31,1.0,7.0,10.0,0.0
18718,2210300004,a_a015688,3,롯데프리미엄아울렛 파주점,2022-10-30,2022-10-30,경기 파주시 문발로 284,경기 파주시 문발동 644,126.692643,37.718115,...,6.0,6.0,경기도,경기,나홀로 여행,2022-10-30~2022-10-31,1.0,7.0,10.0,0.0
28239,2210300003,a_a015688,2,지혜의숲,2022-10-30,2022-10-30,경기 파주시 회동길 145,경기 파주시 문발동 524-3,126.687060,37.708354,...,6.0,6.0,경기도,경기,나홀로 여행,2022-10-30~2022-10-31,1.0,7.0,10.0,0.0


In [48]:
# 전처리 1
df_fil = df[~df['TRAVEL_MISSION_CHECK'].isnull()].copy()
df_fil.loc[:, 'TRAVEL_MISSION_INT'] = df_fil['TRAVEL_MISSION_CHECK'].str.split(';').str[0].astype(int)
df_fil #23898 rows × 68 columns

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,VISIT_START_YMD,VISIT_END_YMD,ROAD_NM_ADDR,LOTNO_ADDR,X_COORD,Y_COORD,...,TRAVEL_STYL_8,TRAVEL_STATUS_RESIDENCE,TRAVEL_STATUS_DESTINATION,TRAVEL_STATUS_ACCOMPANY,TRAVEL_STATUS_YMD,TRAVEL_MOTIVE_1,TRAVEL_MOTIVE_2,TRAVEL_MOTIVE_3,TRAVEL_COMPANIONS_NUM,TRAVEL_MISSION_INT
0,2210300006,a_a015688,5,프로방스마을,2022-10-30,2022-10-30,경기 파주시 탄현면 새오리로 77,경기 파주시 탄현면 성동리 82-1,126.685239,37.790405,...,6.0,경기도,경기,나홀로 여행,2022-10-30~2022-10-31,1.0,7.0,10.0,0.0,5
1,2208050001,a_a000164,1,친구/친지집,2022-08-05,2022-08-05,NaN,경기도 안산시 단원구 와동,NaN,NaN,...,5.0,경기도,경기,2인 여행(가족 외),2022-08-05~2022-08-06,3.0,2.0,1.0,1.0,22
2,2210030002,a_a004293,2,병점역 1호선,2022-10-03,2022-10-03,경기 화성시 떡전골로 97,경기 화성시 진안동 824-1,127.033200,37.206753,...,2.0,경기도,경기,나홀로 여행,2022-10-03~2022-10-03,6.0,2.0,1.0,0.0,28
3,2208200003,a_a000491,12,더현대서울,2022-08-20,2022-08-20,서울 영등포구 여의대로 108,서울 영등포구 여의도동 22,126.928446,37.525872,...,7.0,울산광역시,서울,2인 가족 여행,2022-08-19~2022-08-20,1.0,3.0,7.0,1.0,6
4,2208110007,a_a000172,13,강릉중앙시장,2022-08-11,2022-08-11,강원 강릉시 금성로 21,강원 강릉시 성남동 50,128.898624,37.754025,...,4.0,광주광역시,서울,나홀로 여행,2022-08-10~2022-08-13,1.0,7.0,2.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30149,2210220005,a_a008553,5,집,2022-10-22,2022-10-23,NaN,NaN,NaN,NaN,...,3.0,서울특별시,경기,나홀로 여행,2022-10-22~2022-10-22,2.0,1.0,NaN,0.0,6
30150,2208280003,a_a001505,6,카페아쁠뤼스,2022-08-28,2022-08-28,경기 가평군 청평면 신청평로 107,경기 가평군 청평면 대성리 3-5,127.406939,37.728238,...,4.0,서울특별시,경기,3인 이상 여행(가족 외),2022-08-27~2022-08-28,1.0,3.0,7.0,7.0,5
30151,2210290005,a_a013660,4,국립중앙박물관,2022-10-29,2022-10-29,서울 용산구 서빙고로 137,서울 용산구 용산동6가 168-6,126.980203,37.523847,...,6.0,경기도,서울,나홀로 여행,2022-10-29~2022-10-30,1.0,7.0,8.0,0.0,3
30152,2208060008,a_a000019,7,송도달빛축제공원,2022-08-06,2022-08-06,인천 연수구 센트럴로 350,인천 연수구 송도동 26-1,126.634896,37.407218,...,3.0,서울특별시,인천,3인 이상 여행(가족 외),2022-08-06~2022-08-07,7.0,5.0,1.0,6.0,9


In [29]:
df_fil.iloc[:,20:40]

,REVISIT_INTENTION,RCMDTN_INTENTION,SGG_CD,TRAVEL_NM,TRAVELER_ID,TRAVEL_PURPOSE,TRAVEL_START_YMD,TRAVEL_END_YMD,MVMN_NM,TRAVEL_PERSONA,TRAVEL_MISSION,TRAVEL_MISSION_CHECK,RESIDENCE_SGG_CD,GENDER,AGE_GRP,EDU_NM,EDU_FNSH_SE,MARR_STTS,FAMILY_MEMB,JOB_NM
0,4.0,4.0,NaN,A03,a015688,6;1;,2022-10-30,2022-11-01,대중교통 등,NaN,6;1;,5;6;1,41.0,남,30.0,6.0,1.0,2.0,3.0,2.0
1,NaN,NaN,41273.0,A03,a000164,10;21;22;24;27;,2022-08-05,2022-08-07,자가용,NaN,2;,22;24;1,41.0,남,40.0,6.0,1.0,1.0,1.0,2.0
2,5.0,5.0,NaN,A03,a004293,21;28;,2022-10-03,2022-10-04,NaN,NaN,21;28;,28;3;21,41.0,여,40.0,6.0,1.0,2.0,3.0,2.0
3,5.0,5.0,NaN,A01,a000491,1;22;23;6;,2022-08-19,2022-08-21,자가용,NaN,1;22;23;6;,6;23;1,31.0,여,30.0,6.0,1.0,2.0,3.0,NaN
4,4.0,4.0,NaN,A01,a000172,2;6;,2022-08-10,2022-08-14,대중교통 등,NaN,2;,2;6;1,29.0,남,30.0,6.0,1.0,1.0,5.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30149,NaN,NaN,NaN,A03,a008553,6;1;22;,2022-10-22,2022-10-23,대중교통 등,NaN,6;1;22;,6;1;22,11.0,여,20.0,6.0,1.0,1.0,1.0,2.0
30150,4.0,4.0,NaN,A03,a001505,5;,2022-08-27,2022-08-29,자가용,수도권 거주 39세 이하 3인 이상 친구,5;,5;8;22,11.0,여,30.0,6.0,1.0,1.0,1.0,2.0
30151,5.0,5.0,NaN,A01,a013660,1;3;2;22;,2022-10-29,2022-10-31,대중교통 등,NaN,1;3;2;22;,3;6;22,41.0,남,20.0,6.0,1.0,1.0,1.0,2.0
30152,4.0,4.0,NaN,A02,a000019,22;4;6;7;9;,2022-08-06,2022-08-08,대중교통 등,수도권 거주 39세 이하 3인 이상 친구,2;,9;2;1,11.0,남,20.0,6.0,1.0,1.0,1.0,3.0


In [35]:
df_fil.iloc[:,40:60]

,JOB_ETC,INCOME,HOUSE_INCOME,TRAVEL_TERM,TRAVEL_NUM,TRAVEL_LIKE_SIDO_1,TRAVEL_LIKE_SGG_1,TRAVEL_LIKE_SIDO_2,TRAVEL_LIKE_SGG_2,TRAVEL_LIKE_SIDO_3,TRAVEL_LIKE_SGG_3,TRAVEL_STYL_1,TRAVEL_STYL_2,TRAVEL_STYL_3,TRAVEL_STYL_4,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_STATUS_RESIDENCE
0,NaN,6.0,6.0,3.0,6.0,41.0,41480.0,45.0,45110.0,27.0,27230.0,4.0,4.0,2.0,4.0,6.0,5.0,6.0,6.0,경기도
1,NaN,5.0,NaN,3.0,4.0,42.0,42150.0,26.0,26350.0,41.0,41670.0,1.0,4.0,1.0,3.0,3.0,5.0,3.0,5.0,경기도
2,NaN,8.0,12.0,3.0,2.0,50.0,50110.0,42.0,42820.0,26.0,26350.0,4.0,3.0,6.0,3.0,2.0,2.0,7.0,2.0,경기도
3,1.0,5.0,7.0,4.0,1.0,26.0,26350.0,47.0,47130.0,48.0,48310.0,4.0,4.0,4.0,3.0,4.0,4.0,5.0,7.0,울산광역시
4,NaN,4.0,7.0,3.0,1.0,50.0,50130.0,26.0,26350.0,46.0,46130.0,7.0,1.0,1.0,1.0,7.0,7.0,2.0,4.0,광주광역시
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30149,NaN,4.0,NaN,3.0,4.0,42.0,42150.0,26.0,26350.0,45.0,45110.0,4.0,3.0,4.0,3.0,1.0,3.0,6.0,3.0,서울특별시
30150,NaN,3.0,NaN,3.0,2.0,50.0,50130.0,26.0,26350.0,41.0,41820.0,1.0,3.0,1.0,3.0,4.0,4.0,5.0,4.0,서울특별시
30151,NaN,5.0,NaN,3.0,2.0,11.0,11710.0,26.0,26530.0,50.0,50110.0,3.0,6.0,2.0,5.0,4.0,4.0,5.0,6.0,경기도
30152,NaN,6.0,NaN,3.0,2.0,26.0,26350.0,42.0,42210.0,50.0,50130.0,5.0,2.0,2.0,4.0,4.0,5.0,5.0,3.0,서울특별시


In [34]:
df_fil.iloc[:,60:-1]

,TRAVEL_STATUS_DESTINATION,TRAVEL_STATUS_ACCOMPANY,TRAVEL_STATUS_YMD,TRAVEL_MOTIVE_1,TRAVEL_MOTIVE_2,TRAVEL_MOTIVE_3,TRAVEL_COMPANIONS_NUM
0,경기,나홀로 여행,2022-10-30~2022-10-31,1.0,7.0,10.0,0.0
1,경기,2인 여행(가족 외),2022-08-05~2022-08-06,3.0,2.0,1.0,1.0
2,경기,나홀로 여행,2022-10-03~2022-10-03,6.0,2.0,1.0,0.0
3,서울,2인 가족 여행,2022-08-19~2022-08-20,1.0,3.0,7.0,1.0
4,서울,나홀로 여행,2022-08-10~2022-08-13,1.0,7.0,2.0,0.0
...,...,...,...,...,...,...,...
30149,경기,나홀로 여행,2022-10-22~2022-10-22,2.0,1.0,NaN,0.0
30150,경기,3인 이상 여행(가족 외),2022-08-27~2022-08-28,1.0,3.0,7.0,7.0
30151,서울,나홀로 여행,2022-10-29~2022-10-30,1.0,7.0,8.0,0.0
30152,인천,3인 이상 여행(가족 외),2022-08-06~2022-08-07,7.0,5.0,1.0,6.0


In [22]:
df_fil.columns

Index(['VISIT_AREA_ID', 'TRAVEL_ID', 'VISIT_ORDER', 'VISIT_AREA_NM',
       'VISIT_START_YMD', 'VISIT_END_YMD', 'ROAD_NM_ADDR', 'LOTNO_ADDR',
       'X_COORD', 'Y_COORD', 'ROAD_NM_CD', 'LOTNO_CD', 'POI_ID', 'POI_NM',
       'RESIDENCE_TIME_MIN', 'VISIT_AREA_TYPE_CD', 'REVISIT_YN',
       'VISIT_CHC_REASON_CD', 'LODGING_TYPE_CD', 'DGSTFN', 'REVISIT_INTENTION',
       'RCMDTN_INTENTION', 'SGG_CD', 'TRAVEL_NM', 'TRAVELER_ID',
       'TRAVEL_PURPOSE', 'TRAVEL_START_YMD', 'TRAVEL_END_YMD', 'MVMN_NM',
       'TRAVEL_PERSONA', 'TRAVEL_MISSION', 'TRAVEL_MISSION_CHECK',
       'RESIDENCE_SGG_CD', 'GENDER', 'AGE_GRP', 'EDU_NM', 'EDU_FNSH_SE',
       'MARR_STTS', 'FAMILY_MEMB', 'JOB_NM', 'JOB_ETC', 'INCOME',
       'HOUSE_INCOME', 'TRAVEL_TERM', 'TRAVEL_NUM', 'TRAVEL_LIKE_SIDO_1',
       'TRAVEL_LIKE_SGG_1', 'TRAVEL_LIKE_SIDO_2', 'TRAVEL_LIKE_SGG_2',
       'TRAVEL_LIKE_SIDO_3', 'TRAVEL_LIKE_SGG_3', 'TRAVEL_STYL_1',
       'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5',
      

In [46]:
a = df_fil.isna().sum()
a[41:60]

,0
INCOME,4476
HOUSE_INCOME,10008
TRAVEL_TERM,4476
TRAVEL_NUM,4476
TRAVEL_LIKE_SIDO_1,4476
TRAVEL_LIKE_SGG_1,4476
TRAVEL_LIKE_SIDO_2,4476
TRAVEL_LIKE_SGG_2,4476
TRAVEL_LIKE_SIDO_3,4476
TRAVEL_LIKE_SGG_3,4476


### 필요한 피쳐
- GENDER: 성별
- AGE_GRP: 연령대
- TRAVEL_STYL_1 ~ TRAVEL_STYL_8: 여행성향
- TRAVEL_MOTIVE_1: 여행동기(첫번째만)
- TRAVEL_COMPANIONS_NUM: 동행인 수
- TRAVEL_MISSION_INT: 미션 수행 중요도(첫번째만)
- VISIT_AREA_NM: 방문 지역명
- DGSTFN: 만족도
------------------
- TRAVEL_PURPOSE: 목적
- TRAVEL_TERM: 여행 기간

In [62]:
df_fil[['TRAVEL_PURPOSE', 'TRAVEL_MOTIVE_1']]

,TRAVEL_PURPOSE,TRAVEL_MOTIVE_1
0,6;1;,1.0
1,10;21;22;24;27;,3.0
2,21;28;,6.0
3,1;22;23;6;,1.0
4,2;6;,1.0
...,...,...
30149,6;1;22;,2.0
30150,5;,1.0
30151,1;3;2;22;,1.0
30152,22;4;6;7;9;,7.0


In [81]:
# 전처리 1
df_fil2 = df_fil[~df_fil['TRAVEL_PURPOSE'].isnull()].copy()
df_fil2.loc[:, 'TRAVEL_PURPOSE_INT'] = df_fil2['TRAVEL_PURPOSE'].str.split(';').str[0].astype(int)
df_fil2.iloc[:,-1] #23898 rows × 68 columns

,TRAVEL_PURPOSE_INT
0,6
1,10
2,21
3,1
4,2
...,...
30149,6
30150,5
30151,1
30152,22


In [82]:
# 전처리 2
df_fil2 = df_fil2[[
    'GENDER', 'AGE_GRP',
    'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
    'TRAVEL_MOTIVE_1',
    'TRAVEL_COMPANIONS_NUM',
    'TRAVEL_MISSION_INT',
    'VISIT_AREA_NM',
    'TRAVEL_PURPOSE_INT',
    'TRAVEL_TERM',
    'DGSTFN', # 만족도
]]

df_fil2 = df_fil2.dropna()
df_fil2 # 14930 rows × 15 columns

,GENDER,AGE_GRP,TRAVEL_STYL_1,TRAVEL_STYL_2,TRAVEL_STYL_3,TRAVEL_STYL_4,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_MOTIVE_1,TRAVEL_COMPANIONS_NUM,TRAVEL_MISSION_INT,VISIT_AREA_NM,TRAVEL_PURPOSE_INT,TRAVEL_TERM,DGSTFN
0,남,30.0,4.0,4.0,2.0,4.0,6.0,5.0,6.0,6.0,1.0,0.0,5,프로방스마을,6,3.0,4.0
2,여,40.0,4.0,3.0,6.0,3.0,2.0,2.0,7.0,2.0,6.0,0.0,28,병점역 1호선,21,3.0,4.0
3,여,30.0,4.0,4.0,4.0,3.0,4.0,4.0,5.0,7.0,1.0,1.0,6,더현대서울,1,4.0,5.0
4,남,30.0,7.0,1.0,1.0,1.0,7.0,7.0,2.0,4.0,1.0,0.0,2,강릉중앙시장,2,3.0,4.0
5,남,20.0,6.0,1.0,1.0,3.0,4.0,6.0,3.0,2.0,3.0,1.0,11,청계천,1,2.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30145,여,30.0,3.0,1.0,3.0,3.0,4.0,4.0,7.0,6.0,1.0,0.0,3,행궁동벽화마을,1,3.0,2.0
30150,여,30.0,1.0,3.0,1.0,3.0,4.0,4.0,5.0,4.0,1.0,7.0,5,카페아쁠뤼스,5,3.0,5.0
30151,남,20.0,3.0,6.0,2.0,5.0,4.0,4.0,5.0,6.0,1.0,0.0,3,국립중앙박물관,1,3.0,5.0
30152,남,20.0,5.0,2.0,2.0,4.0,4.0,5.0,5.0,3.0,7.0,6.0,9,송도달빛축제공원,22,3.0,4.0


In [83]:
df_fil2['TRAVEL_COMPANIONS_NUM'].unique()

array([ 0.,  1.,  2.,  3.,  4., 11.,  5.,  7., 14., 19.,  6., 20.,  9.,
       13.,  8.])

In [84]:
# 전처리 3
cat_features_names = [
    'GENDER',
    #'AGE_GRP',
    'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
    'TRAVEL_MOTIVE_1',
    #'TRAVEL_COMPANIONS_NUM',
    'TRAVEL_MISSION_INT',
    'VISIT_AREA_NM',
    'TRAVEL_PURPOSE_INT',
    #'TRAVEL_TERM',
    #'DGSTFN',
]
int_convert_cols = [
    'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4',
    'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
    'TRAVEL_MOTIVE_1', 'TRAVEL_MISSION_INT', 'TRAVEL_PURPOSE_INT'
]

df_fil2[int_convert_cols] = df_fil2[int_convert_cols].astype(int)
df_fil2

,GENDER,AGE_GRP,TRAVEL_STYL_1,TRAVEL_STYL_2,TRAVEL_STYL_3,TRAVEL_STYL_4,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_MOTIVE_1,TRAVEL_COMPANIONS_NUM,TRAVEL_MISSION_INT,VISIT_AREA_NM,TRAVEL_PURPOSE_INT,TRAVEL_TERM,DGSTFN
0,남,30.0,4,4,2,4,6,5,6,6,1,0.0,5,프로방스마을,6,3.0,4.0
2,여,40.0,4,3,6,3,2,2,7,2,6,0.0,28,병점역 1호선,21,3.0,4.0
3,여,30.0,4,4,4,3,4,4,5,7,1,1.0,6,더현대서울,1,4.0,5.0
4,남,30.0,7,1,1,1,7,7,2,4,1,0.0,2,강릉중앙시장,2,3.0,4.0
5,남,20.0,6,1,1,3,4,6,3,2,3,1.0,11,청계천,1,2.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30145,여,30.0,3,1,3,3,4,4,7,6,1,0.0,3,행궁동벽화마을,1,3.0,2.0
30150,여,30.0,1,3,1,3,4,4,5,4,1,7.0,5,카페아쁠뤼스,5,3.0,5.0
30151,남,20.0,3,6,2,5,4,4,5,6,1,0.0,3,국립중앙박물관,1,3.0,5.0
30152,남,20.0,5,2,2,4,4,5,5,3,7,6.0,9,송도달빛축제공원,22,3.0,4.0


In [85]:
df_fil2.to_csv('/content/drive/MyDrive/건양대의료인공지능_실습/Travel_recommendation/df_fil2.csv', index=False)

In [86]:
# 데이터셋 분류
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df_fil2, test_size = 0.2, random_state=2023)
print(train_data.shape)
print(test_data.shape)

(11944, 17)
(2986, 17)


In [73]:
!pip install -q catboost # 범주 분류할때 1티어, 병렬 처리, 교차 검증, gpu 지원, 하이퍼파라미터 조정가능

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.3 MB/s eta 0:00:00


In [87]:
cat_features_names

['GENDER',
 'TRAVEL_STYL_1',
 'TRAVEL_STYL_2',
 'TRAVEL_STYL_3',
 'TRAVEL_STYL_4',
 'TRAVEL_STYL_5',
 'TRAVEL_STYL_6',
 'TRAVEL_STYL_7',
 'TRAVEL_STYL_8',
 'TRAVEL_MOTIVE_1',
 'TRAVEL_MISSION_INT',
 'VISIT_AREA_NM',
 'TRAVEL_PURPOSE_INT']

In [88]:
# Catboost
from catboost import CatBoostRegressor, Pool
train_pool = Pool(train_data.drop(['DGSTFN'], axis=1),# 열방향
                  label = train_data['DGSTFN'],
                  cat_features = cat_features_names)

test_pool = Pool(test_data.drop(['DGSTFN'], axis=1),# 열방향
                 label = test_data['DGSTFN'],
                 cat_features = cat_features_names)

In [103]:
train_data.drop(['DGSTFN'], axis=1)

,GENDER,AGE_GRP,TRAVEL_STYL_1,TRAVEL_STYL_2,TRAVEL_STYL_3,TRAVEL_STYL_4,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_MOTIVE_1,TRAVEL_COMPANIONS_NUM,TRAVEL_MISSION_INT,VISIT_AREA_NM,TRAVEL_PURPOSE_INT,TRAVEL_TERM
13070,남,40.0,2,1,4,4,4,3,2,2,8,3.0,3,경복궁,3,2.0
1272,여,30.0,4,1,3,2,2,4,4,7,3,2.0,2,두타몰,1,2.0
20137,여,20.0,1,1,1,4,1,1,1,7,2,1.0,25,문교오리랜드,1,2.0
9928,남,30.0,6,1,4,2,2,4,3,5,1,1.0,9,화담숲,10,2.0
13365,남,40.0,1,1,2,2,6,3,5,6,3,5.0,7,청춘조개 본점,1,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12191,여,20.0,3,2,2,4,5,3,6,5,9,2.0,1,원조누드치즈김밥,1,3.0
5576,여,30.0,1,1,3,4,2,2,2,6,3,3.0,1,이화여자대학교,1,3.0
29863,남,30.0,4,2,2,3,4,4,6,4,2,0.0,21,화성행궁,1,3.0
11420,여,20.0,4,3,3,4,4,5,5,5,4,0.0,6,더백푸드트럭 해방촌점,1,3.0


In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [89]:
# 모델 생성
model = CatBoostRegressor(
    loss_function = 'RMSE', # 평균제곱근오차
    eval_metric='MAE', # 평균오차
    task_type='CPU', #GPU 변경가능
    depth=6,
    learning_rate=0.01,
    n_estimators=2000)

model.fit(
    train_pool,
    eval_set = test_pool,
    verbose=500,
    plot=True )

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6848667	test: 0.6995523	best: 0.6995523 (0)	total: 149ms	remaining: 4m 57s
500:	learn: 0.6090137	test: 0.6151520	best: 0.6151520 (500)	total: 21.2s	remaining: 1m 3s
1000:	learn: 0.5990863	test: 0.6093590	best: 0.6093590 (1000)	total: 46.4s	remaining: 46.3s
1500:	learn: 0.5919787	test: 0.6054897	best: 0.6054875 (1498)	total: 1m 7s	remaining: 22.5s
1999:	learn: 0.5856950	test: 0.6035313	best: 0.6035313 (1999)	total: 1m 32s	remaining: 0us

bestTest = 0.6035313291
bestIteration = 1999



In [109]:
model.save_model('/content/catboost_model.cbm')

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

In [91]:
test_data.iloc[5]

,20675
GENDER,남
AGE_GRP,30.0
TRAVEL_STYL_1,1
TRAVEL_STYL_2,4
TRAVEL_STYL_3,7
TRAVEL_STYL_4,7
TRAVEL_STYL_5,4
TRAVEL_STYL_6,1
TRAVEL_STYL_7,7
TRAVEL_STYL_8,1


In [94]:
# 모델 테스트
model.predict(test_data.iloc[13].drop(['DGSTFN']))

np.float64(4.513293356883928)

In [95]:
# 테스트데이터 갯수 확인 (1~5점) # 만족도 지표가 1, 2은 거의 없고
test_data[test_data['DGSTFN'] == 4].index

Index([20824,  6268, 20675, 29428,  4806,  7370, 11277, 10804, 27631,   580,
       ...
        9408, 26517,  6809, 29131,  5230, 17484, 17505,  5242,  3555, 18105],
      dtype='int64', length=1115)

In [96]:
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,TRAVEL_MISSION_INT,12.499748
1,VISIT_AREA_NM,10.238666
2,TRAVEL_STYL_6,8.793752
3,TRAVEL_STYL_8,8.557618
4,TRAVEL_STYL_4,8.399562
5,TRAVEL_MOTIVE_1,7.821628
6,TRAVEL_PURPOSE_INT,7.268445
7,TRAVEL_STYL_1,7.116619
8,TRAVEL_STYL_2,7.101793
9,TRAVEL_STYL_7,7.087292


In [97]:
area_names = df_fil2[['VISIT_AREA_NM']].drop_duplicates()
area_names

,VISIT_AREA_NM
0,프로방스마을
2,병점역 1호선
3,더현대서울
4,강릉중앙시장
5,청계천
...,...
30128,설악버스터미널
30136,탄도어촌체험마을
30138,하남ICEBOX
30141,부평역사박물관


In [107]:
traveler = {
    'GENDER':'여',
    'AGE_GRP' : 26.0,
    'TRAVEL_STYL_1' : 2,
    'TRAVEL_STYL_2' : 1,
    'TRAVEL_STYL_3' : 1,
    'TRAVEL_STYL_4' : 4,
    'TRAVEL_STYL_5' : 1,
    'TRAVEL_STYL_6' : 4,
    'TRAVEL_STYL_7' : 2,
    'TRAVEL_STYL_8' : 6,
    'TRAVEL_MOTIVE_1' : 3,
    'TRAVEL_COMPANIONS_NUM' : 4.0,
    'TRAVEL_MISSION_INT' : 5,
    'TRAVEL_PURPOSE_INT' : 6,
    'TRAVEL_TERM' : 2
     }
results = pd.DataFrame(columns=['AREA', 'SCORE'])

for area in area_names['VISIT_AREA_NM']:
    input_data = traveler.copy()
    input_data['VISIT_AREA_NM'] = area  # 지역 정보 추가

    # DataFrame으로 변환
    input_df = pd.DataFrame([input_data])

    # 문자열이 아닌 cat_features 컬럼을 string으로 강제 변환 (안전성 확보)
    for col in cat_features_names:
        input_df[col] = input_df[col].astype(str)

    # Pool로 감싸서 예측
    input_pool = Pool(input_df, cat_features=cat_features_names)
    score = model.predict(input_pool)

    results = pd.concat([results, pd.DataFrame([[area, score[0]]], columns=['AREA', 'SCORE'])])

# 결과 상위 10개
results.sort_values('SCORE', ascending=False).head(10)

<ipython-input-107-9bae9f600e02>:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, pd.DataFrame([[area, score[0]]], columns=['AREA', 'SCORE'])])


,AREA,SCORE
0,소피텔 앰배서더 서울,4.471451
0,송도센트럴파크,4.460326
0,덕수궁,4.449059
0,용산역,4.436285
0,스타필드 하남,4.433902
0,국립현대미술관 서울관,4.422280
0,수서역,4.421874
0,남산서울타워,4.421688
0,화담숲,4.420441
0,안다즈 서울강남,4.420302


In [108]:
results.sort_values('SCORE', ascending=True)[:10]

,AREA,SCORE
0,용문막국수,4.074366
0,스파오 라붐서울대점,4.093422
0,저스트텐동 행리단길점,4.096345
0,쉬자파크 치유의집,4.096624
0,브라운스마일,4.096624
0,남산공원,4.098189
0,골드식당 본점,4.100035
0,스타벅스 경복궁사거리점,4.100035
0,십원집 봉천점,4.100035
0,카리스호텔,4.100035


In [3]:
# TSY
tsy = pd.read_csv('/content/drive/MyDrive/건양대의료인공지능_실습/Travel_recommendation/tc_codeb_B.csv')
tsy = tsy[173:179]
tsy

,idx,cd_nm,cd_a,cd_b,cd_memo,cd_memo2,del_flag,order_num,ins_dt,edit_dt
173,1165,자연 선호 매우선호,TSY,1,NaN,NaN,N,999,2022-09-22 17:34:50.000,NaN
174,1166,자연 선호 중간선호,TSY,2,NaN,NaN,N,999,2022-09-22 17:34:50.000,NaN
175,1167,자연 선호 약간선호,TSY,3,NaN,NaN,N,999,2022-09-22 17:34:50.000,NaN
176,1168,중립,TSY,4,NaN,NaN,N,999,2022-09-22 17:34:50.000,NaN
177,1169,도시 선호 약간선호,TSY,5,NaN,NaN,N,999,2022-09-22 17:34:50.000,NaN
178,1170,도시 선호 중간선호,TSY,6,NaN,NaN,N,999,2022-09-22 17:34:50.000,NaN


In [4]:
# TMT
tmt = pd.read_csv('/content/drive/MyDrive/건양대의료인공지능_실습/Travel_recommendation/tc_codeb_B.csv')
tmt = tmt[180:190]
tmt

,idx,cd_nm,cd_a,cd_b,cd_memo,cd_memo2,del_flag,order_num,ins_dt,edit_dt
180,1172,"일상적인 환경 및 역할에서의 탈출, 지루함 탈피",TMT,1,NaN,NaN,N,999,2022-09-22 17:34:50.000,NaN
181,1173,"쉴 수 있는 기회, 육체 피로 해결 및 정신적인 휴식",TMT,2,NaN,NaN,N,999,2022-09-22 17:34:50.000,NaN
182,1174,여행 동반자와의 친밀감 및 유대감 증진,TMT,3,NaN,NaN,N,999,2022-09-22 17:34:50.000,NaN
183,1175,진정한 자아 찾기 또는 자신을 되돌아볼 기회 찾기,TMT,4,NaN,NaN,N,999,2022-09-22 17:34:50.000,NaN
184,1176,SNS 사진 등록 등 과시,TMT,5,NaN,NaN,N,999,2022-09-22 17:34:50.000,NaN
185,1177,"운동, 건강 증진 및 충전",TMT,6,NaN,NaN,N,999,2022-09-22 17:34:50.000,NaN
186,1178,새로운 경험 추구,TMT,7,NaN,NaN,N,999,2022-09-22 17:34:50.000,NaN
187,1179,"역사 탐방, 문화적 경험 등 교육적 동기",TMT,8,NaN,NaN,N,999,2022-09-22 17:34:50.000,NaN
188,1180,"특별한 목적(칠순여행, 신혼여행, 수학여행, 인센티브여행)",TMT,9,NaN,NaN,N,999,2022-09-22 17:34:50.000,NaN
189,1181,기타,TMT,10,NaN,NaN,N,999,2022-09-22 17:34:50.000,NaN


In [5]:
# MIS
mis = pd.read_csv('/content/drive/MyDrive/건양대의료인공지능_실습/Travel_recommendation/tc_codeb_B.csv')
mis = mis[105:126]
mis

,idx,cd_nm,cd_a,cd_b,cd_memo,cd_memo2,del_flag,order_num,ins_dt,edit_dt
105,1097,쇼핑,MIS,1,NaN,NaN,N,999,2022-07-27 16:26:52.000,NaN
106,1098,"테마파크, 놀이시설, 동/식물원 방문",MIS,2,NaN,NaN,N,999,2022-07-27 16:26:58.000,NaN
107,1099,역사 유적지 방문,MIS,3,NaN,NaN,N,999,2022-07-27 16:27:03.000,NaN
108,1100,시티투어,MIS,4,NaN,NaN,N,999,2022-07-27 16:27:08.000,NaN
109,1101,"야외 스포츠, 레포츠 활동",MIS,5,NaN,NaN,N,999,2022-07-27 16:27:14.000,NaN
...,...,...,...,...,...,...,...,...,...,...
121,1113,신규 여행지 발굴,MIS,24,NaN,NaN,N,999,2022-07-27 16:28:21.000,NaN
122,1114,반려동물 동반 여행,MIS,25,NaN,NaN,N,999,2022-07-27 16:28:26.000,NaN
123,1115,인플루언서 따라하기 여행,MIS,26,NaN,NaN,N,999,2022-07-27 16:28:34.000,NaN
124,1116,친환경 여행(플로깅 여행),MIS,27,NaN,NaN,N,999,2022-07-27 16:30:10.000,NaN
